<a href="https://colab.research.google.com/github/Ernawaty2024/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [ ]:
# Check the variance of each numerical column
for column in application_df.select_dtypes(include=['number']).columns:
    print(f"{column}: {application_df[column].var()}")

STATUS: 0.0001457598435995093
ASK_AMT: 7591715741692320.0
IS_SUCCESSFUL: 0.24895709711716982


In [ ]:
# Determine the number of unique values in each column.
print(application_df.nunique().to_string(header=False))


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2


### Preprocessing 'ASK_AMT' column to bin it


In [ ]:
# Find minimun value of ASK_AMT
min_ask_amt = application_df['ASK_AMT'].min()
print(min_ask_amt)

5000


In [ ]:
# Find maximum value of ASK_AMT
max_ask_amt = application_df['ASK_AMT'].max()
print(max_ask_amt)

8597806340


In [ ]:
# Define bin_edges
bin_edges = [min_ask_amt,5000,15000,50000, 100000,500000,1000000,5000000, max_ask_amt]

# Bin the 'ASK_AMT' column
application_df['ASK_AMT_BIN'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, include_lowest=True, duplicates='drop')

# Display value counts for the binned column
print(application_df['ASK_AMT_BIN'].value_counts().to_string(header=False))

(4999.999, 15000.0]          26334
(100000.0, 500000.0]          2304
(15000.0, 50000.0]            2011
(50000.0, 100000.0]           1423
(1000000.0, 5000000.0]         940
(500000.0, 1000000.0]          650
(5000000.0, 8597806340.0]      637


In [ ]:
application_df = application_df.drop(columns = ['ASK_AMT'])

In [ ]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'IS_SUCCESSFUL', 'ASK_AMT_BIN'],
      dtype='object')

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts.to_string(header=False))

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_counts[application_counts <500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_value_counts.to_string(header=False))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_to_replace = classification_value_counts[classification_value_counts > 1]
classification_to_replace

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
# Include the 'ASK_AMT_BIN' column in the get_dummies call
categorical_dummies = pd.get_dummies(application_df,
                                      columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT_BIN',
                                               'AFFILIATION', 'USE_CASE', 'ORGANIZATION',
                                               'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], drop_first=True)


In [ ]:
# Split our preprocessed data into our features and target arrays
X = categorical_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = categorical_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, random_state = 42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the custom threshold callback
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs["accuracy"]  # or "val_accuracy" for validation accuracy
        if accuracy >= self.threshold:
            self.model.stop_training = True

# Ensure the directory for saving model weights exists
checkpoint_dir = "checkpoints/"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [ ]:
# Define the checkpoint callback to save weights every 5 epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "weights_epoch_{epoch:02d}.weights.h5"),  # Changed the filename
    save_weights_only=True,
    verbose=1,
    save_best_only=True,  # Save only the best model
    save_freq=5  # Save every 5 epochs based on batches
)


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 80)                  │           3,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,821 (22.74 KB)

 Trainable params: 5,821 (22.74 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Define the custom callback
my_callback = MyThresholdCallback(threshold=0.75)

# Train the model with the custom callback
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[my_callback])



Epoch 1/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6842 - loss: 0.6069 - val_accuracy: 0.7198 - val_loss: 0.5702
Epoch 2/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7280 - loss: 0.5574 - val_accuracy: 0.7157 - val_loss: 0.5670
Epoch 3/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7314 - loss: 0.5484 - val_accuracy: 0.7181 - val_loss: 0.5657
Epoch 4/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7356 - loss: 0.5436 - val_accuracy: 0.7201 - val_loss: 0.5647
Epoch 5/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7309 - loss: 0.5506 - val_accuracy: 0.7216 - val_loss: 0.5656
Epoch 6/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7282 - loss: 0.5510 - val_accuracy: 0.7213 - val_loss: 0.5648
Epoch 7/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7366 - loss: 0.5432 - val_accuracy: 0.7198 - val_loss: 0.5649
Epoch 8/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7339 - loss: 0.5436 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - 1ms/step - accuracy: 0.7149 - loss: 0.5747
Loss: 0.5746772885322571, Accuracy: 0.7148687839508057


# Compile, Train and Optimize the Model for first optimization
- add another hidden layer



In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid',))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units = 10, activation = 'tanh'))

# Output layer
nn1.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn1.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                     │ (None, 80)                  │           3,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 10)                  │             310 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,111 (23.87 KB)

 Trainable params: 6,111 (23.87 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
stop_early = EarlyStopping(monitor='val_accuracy', patience=100, mode='max', baseline=0.75)

# Train the model with the EarlyStopping callback
model_train = nn1.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks = [stop_early])

Epoch 1/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7045 - loss: 0.5930 - val_accuracy: 0.7227 - val_loss: 0.5692
Epoch 2/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7299 - loss: 0.5526 - val_accuracy: 0.7254 - val_loss: 0.5653
Epoch 3/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7314 - loss: 0.5500 - val_accuracy: 0.7219 - val_loss: 0.5636
Epoch 4/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7319 - loss: 0.5466 - val_accuracy: 0.7262 - val_loss: 0.5625
Epoch 5/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7312 - loss: 0.5492 - val_accuracy: 0.7219 - val_loss: 0.5653
Epoch 6/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7314 - loss: 0.5505 - val_accuracy: 0.7198 - val_loss: 0.5664
Epoch 7/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7349 - loss: 0.5445 - val_accuracy: 0.7219 - val_loss: 0.5643
Epoch 8/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7348 - loss: 0.5443 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7299 - loss: 0.5617
Loss: 0.5616850256919861, Accuracy: 0.729912519454956


## Second Optimization
- Increase epoch from 100 to 200


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid'))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_29 (Dense)                     │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_train = nn2.fit(X_train_scaled, y_train, epochs=200, validation_data=(X_test_scaled, y_test))

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7016 - loss: 0.5927 - val_accuracy: 0.7230 - val_loss: 0.5654
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7310 - loss: 0.5549 - val_accuracy: 0.7270 - val_loss: 0.5573
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7281 - loss: 0.5548 - val_accuracy: 0.7287 - val_loss: 0.5561
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7273 - loss: 0.5552 - val_accuracy: 0.7292 - val_loss: 0.5544
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7302 - loss: 0.5515 - val_accuracy: 0.7273 - val_loss: 0.5550
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7330 - loss: 0.5447 - val_accuracy: 0.7268 - val_loss: 0.5534
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7301 - loss: 0.5500 - val_accuracy: 0.7319 - val_loss: 0.5539
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7332 - loss: 0.5486 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7256 - loss: 0.5642
Loss: 0.5641893744468689, Accuracy: 0.7255976796150208


# Third Optimization
- using hyperparameter Tuning


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt

# Define the EarlyStopping callback and assing it to stop_early
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.001,
    verbose=0,
    mode='max',
    baseline=0.75,
    restore_best_weights=True)

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=200,
    factor = 3,
    directory="tune_dir",
    project_name="tune_project")

Reloading Tuner from tune_dir/tune_project/tuner0.json


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test), callbacks=[stop_early])

Trial 246 Complete [00h 00m 19s]
val_accuracy: 0.7266472578048706

Best val_accuracy So Far: 0.7351603507995605
Total elapsed time: 01h 16m 37s


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 49,
 'num_layers': 4,
 'units_0': 23,
 'units_1': 99,
 'units_2': 1,
 'units_3': 83,
 'units_4': 65,
 'units_5': 61,
 'tuner/epochs': 23,
 'tuner/initial_epoch': 8,
 'tuner/bracket': 3,
 'tuner/round': 1,
 'tuner/trial_id': '0175'}

##First Optimization
- Change activation to tanh
- Add more layers


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 39, activation = 'tanh', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 7, activation = 'tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 3, activation = 'tanh'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = 5, activation = 'tanh'))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units = 5, activation = 'tanh'))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units = 3, activation = 'tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 39)                  │           1,716 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 7)                   │             280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 3)                   │              24 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 5)                   │              30 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 3)                   │              18 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,092 (8.17 KB)

 Trainable params: 2,092 (8.17 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_train = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6877 - loss: 0.6106 - val_accuracy: 0.7290 - val_loss: 0.5747
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7279 - loss: 0.5726 - val_accuracy: 0.7259 - val_loss: 0.5642
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7304 - loss: 0.5583 - val_accuracy: 0.7291 - val_loss: 0.5649
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7379 - loss: 0.5517 - val_accuracy: 0.7270 - val_loss: 0.5584
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7282 - loss: 0.5558 - val_accuracy: 0.7266 - val_loss: 0.5572
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7321 - loss: 0.5516 - val_accuracy: 0.7307 - val_loss: 0.5562
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7319 - loss: 0.5487 - val_accuracy: 0.7264 - val_loss: 0.5572
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7329 - loss: 0.5498 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7256 - loss: 0.5584
Loss: 0.5583932995796204, Accuracy: 0.7255976796150208


## Second Optimization
- change the activation of first and second hidden layers to tanh

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = 'tanh', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = 'tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_train = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7056 - loss: 0.5850 - val_accuracy: 0.7258 - val_loss: 0.5597
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7294 - loss: 0.5531 - val_accuracy: 0.7199 - val_loss: 0.5606
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7297 - loss: 0.5530 - val_accuracy: 0.7248 - val_loss: 0.5579
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7276 - loss: 0.5515 - val_accuracy: 0.7284 - val_loss: 0.5541
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7275 - loss: 0.5543 - val_accuracy: 0.7269 - val_loss: 0.5544
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7299 - loss: 0.5465 - val_accuracy: 0.7297 - val_loss: 0.5535
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7294 - loss: 0.5506 - val_accuracy: 0.7247 - val_loss: 0.5550
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7263 - loss: 0.5509 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7266 - loss: 0.5629
Loss: 0.5628715753555298, Accuracy: 0.7266472578048706


## Third Optimization
- Increase the neurons

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 120, activation = 'relu', input_dim = X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 60, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                     │ (None, 120)                 │           5,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 60)                  │           7,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,601 (49.22 KB)

 Trainable params: 12,601 (49.22 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_train = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6946 - loss: 0.5974 - val_accuracy: 0.7241 - val_loss: 0.5615
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7267 - loss: 0.5565 - val_accuracy: 0.7280 - val_loss: 0.5587
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7290 - loss: 0.5530 - val_accuracy: 0.7284 - val_loss: 0.5573
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7317 - loss: 0.5520 - val_accuracy: 0.7264 - val_loss: 0.5560
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7279 - loss: 0.5502 - val_accuracy: 0.7277 - val_loss: 0.5534
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7332 - loss: 0.5473 - val_accuracy: 0.7230 - val_loss: 0.5540
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7298 - loss: 0.5478 - val_accuracy: 0.7290 - val_loss: 0.5528
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7282 - loss: 0.5529 - val_accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7276 - loss: 0.5680
Loss: 0.5680021643638611, Accuracy: 0.727580189704895
